## C01L04
#### Import dependencies

In [8]:
import os
from dotenv import load_dotenv
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

## Langchain initialization (L1)

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
OPENAI_MODEL ='gpt-3.5-turbo'
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, streaming=True)
chat.predict('Hi, how are you?')

"Hello! I'm an AI, so I don't have feelings, but I'm here to help you. How can I assist you today?"

## Langchain format (L2)
#### Define template strings

In [8]:
SYSTEM_TEMPLATE = """
As a {role} who answers the questions ultra-concisely using CONTEXT below 
and nothing more and truthfully says "don't know" when the CONTEXT is not enough to give an answer.

context###{context}###
"""

HUMAN_TEMPLATE = """{text}"""


In [10]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_TEMPLATE),
    ("human", HUMAN_TEMPLATE),
])


In [14]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, streaming=True)
llm(chat_prompt.format_messages(role="Python Senior Developer", context='You are an expert in Python theory.', text="Tell me basic types of data in Python."))

AIMessageChunk(content='The basic types of data in Python are integers, floats, strings, booleans, and None.')

## Streaming (L3)

In [22]:
class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"Token: {token}")
        
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, streaming=True)
llm.predict('Hi there', callbacks=[MyCustomHandler()])

Token: 
Token: Hello
Token: !
Token:  How
Token:  can
Token:  I
Token:  assist
Token:  you
Token:  today
Token: ?
Token: 


'Hello! How can I assist you today?'

In [33]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
llm([HumanMessage(content='Hi there')])

Hello! How can I assist you today?

AIMessageChunk(content='Hello! How can I assist you today?')

### Conversation (L5)

In [9]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, callbacks=[StreamingStdOutCallbackHandler()], streaming=True, temperature=0)
memory = ConversationBufferWindowMemory(k=1)
chain = ConversationChain(llm=llm, memory=memory)
questions = ["Hey there! I'm Bartek.", "I love pineapples on the pizza. ", 'I really hate Hawaiian pizza.', "What is my name?"]
repsonses = []
for q in questions:
    print(f'\nUSER: {q}')
    repsonses.append(chain.invoke(q))


USER: Hey there! I'm Bartek.
Hello Bartek! How can I assist you today?
USER: I love pineapples on the pizza. 
That's great to hear, Bartek! Pineapples on pizza can be a divisive topic, but it's all about personal preference. Some people enjoy the sweet and tangy flavor it adds to the pizza, while others prefer a more traditional combination of toppings. It's all about finding what you enjoy! Is there anything else you'd like to discuss?
USER: I really hate Hawaiian pizza.
I understand that not everyone enjoys the combination of pineapples on pizza. Taste preferences can vary greatly from person to person, and it's completely okay to have different opinions. Is there anything else you'd like to talk about?
USER: What is my name?
I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.